In [9]:
import argparse
import os
import pandas as pd

dirs_ = os.listdir('./logs')
dirs_ = ['./logs/'+ dir_ for dir_ in dirs_]

final_dic = {'model': [],'config': [], 'score': [], 'monotone': [], 'screenshot': [], 'unknown': []}
configs = []
scores = []
for dir_ in dirs_:
#     print(dir_, '\n\n==============================\n')
    with open(dir_, 'r', encoding="utf8") as f :
        content = f.readlines()
        
        config_enter = 0
        config = ""
        
        score_count = 0
        scores = []
        monotones = []
        screenshots = []
        unknowns = []
        for i, line in enumerate(content):
            # line = line.replace('\n', '')
            if 'Loaded!' in line:
                final_dic['model'].append(line.split(' ')[0])

            if 'batch_size' in line:
                config_enter = 1
                config += line
            
            elif 'weight_decay' in line:
                config_enter = 0
                config += line
                
                final_dic['config'].append(config)
                config = ""
            
            elif config_enter:
                config += line
            
            if 'Final' in line:
                line = line.split(' ')
                scores.append(float(line[3]))
                monotones.append(float(line[5].replace('),', '')))
                screenshots.append(float(line[7].replace('),', '')))
                unknowns.append(float(line[-1].replace(')]\n', '')))

                
            if 'train finished' in line:
                final_dic['score'].append(scores)
                final_dic['monotone'].append(monotones)
                final_dic['screenshot'].append(screenshots)
                final_dic['unknown'].append(unknowns)

                scores = [] ; monotones = [] ; screenshots = [] ; unknowns = [] 

print([len(final_dic[a]) for a in final_dic])
if len(final_dic['config']) > len(final_dic['score']):
    final_dic['config'] = final_dic['config'][:-len(dirs_)]
    final_dic['model'] = final_dic['model'][:-len(dirs_)]
    
    
df = pd.DataFrame.from_dict(final_dic)
df['best_score'] = df['score'].apply(lambda x : max(x))
df['best_idx'] = df[['score', 'best_score']].apply(lambda x: x[0].index(x[1]), axis=1)
df['best_monotone'] = df[['monotone', 'best_idx']].apply(lambda x : x[0][x[1]], axis=1)
df['best_screenshot'] = df[['screenshot', 'best_idx']].apply(lambda x : x[0][x[1]], axis=1)
df['best_unknown'] = df[['unknown', 'best_idx']].apply(lambda x : x[0][x[1]], axis=1)

df = df.sort_values(by=['best_score'])
print("Top 15 models")
df[['model','best_score','best_idx','best_monotone', 'best_screenshot', 'best_unknown']].iloc[-15:].iloc[::-1]


[34, 34, 28, 28, 28, 28]
Top 15 models


,model,best_score,best_idx,best_monotone,best_screenshot,best_unknown
12,VGG16,0.918297,1,0.943005,0.924731,0.954641
17,VGG16,0.915797,1,0.947917,0.930591,0.948539
10,DenseNet121,0.911666,6,0.938776,0.936387,0.946475
24,ResNet50,0.911327,3,0.936842,0.946015,0.943849
13,VGG16,0.910847,2,0.932642,0.926316,0.951150
15,VGG16,0.909721,5,0.936842,0.939633,0.941146
11,VGG16,0.908821,2,0.922280,0.934726,0.947614
14,VGG16,0.908364,8,0.943590,0.911839,0.951888
16,VGG16,0.907803,3,0.916256,0.938144,0.948625
8,DenseNet121,0.907605,5,0.952880,0.942708,0.935117


In [10]:
list_ = df[df['model']=='VGG16'][['config','best_score','score']].iloc[-3:].values.tolist()
list_.reverse()
for l in list_:
    print("Score = %1.5f \n"% l[1], l[2],'\n',l[0])

Score = 0.91830 
 [0.8922865946798396, 0.9182967576755464, 0.9080244064838026, 0.8978565743720566, 0.9099281055041447] 
 {'batch_size': 128,
 'class_ratio': [0.7, 3, 2.5, 1.5],
 'epoch': [2, 15],
 'gamma': 0.1,
 'learning_rate': 1e-05,
 'loss_cls_weight': tensor([1.1000, 1.0000, 1.0000, 1.1000]),
 'optimizer': 'Adam',
 'step_size': 2,
 'transforms': [1, None],
 'val_ratio': [0.03, 0.15, 0.15, 0.2],
 'weight_decay': 0.001}

Score = 0.91580 
 [0.9117308203433817, 0.9157969748570283, 0.903321780832865, 0.9110641372071433, 0.9072205791517128] 
 {'batch_size': 128,
 'class_ratio': [0.7, 3, 2.5, 1.5],
 'epoch': [2, 15],
 'gamma': 0.3,
 'learning_rate': 0.0001,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 2,
 'transforms': [1, None],
 'val_ratio': [0.03, 0.15, 0.15, 0.2],
 'weight_decay': 0.001}

Score = 0.91085 
 [0.8902839719879971, 0.9015129187482624, 0.9108465266231146, 0.9034988690021508, 0.8942095175581182, 0.9051704773197771] 
 {'batch_size': 128,
 

In [11]:
list_ = df[df['model']=='DenseNet121'][['config','best_score','score']].iloc[-3:].values.tolist()
list_.reverse()
for l in list_:
    print("Score = %1.5f \n"% l[1], l[2],'\n',l[0])

Score = 0.91167 
 [0.8928018278154604, 0.8877000025099568, 0.9038703411011939, 0.9029630132300683, 0.9056456913858837, 0.9080911976024804, 0.9116655775672649, 0.9056410598981268, 0.9012915797571891, 0.9067092596303353] 
 {'batch_size': 128,
 'class_ratio': [0.8, 2, 2, 2],
 'epoch': [2, 15],
 'gamma': 0.3,
 'learning_rate': 0.0001,
 'loss_cls_weight': tensor([1.1000, 1.0000, 1.0000, 1.1000]),
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 'optimizer': 'Adam',
 'step_size': 2,
 'transforms': [2,
                Compose(
    ColorJitter(brightness=None, contrast=None, saturation=[0.95, 1.05], hue=[-0.05, 0.05])
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)],

In [12]:
list_ = df[df['model']=='ResNet50'][['config','best_score','score']].iloc[-3:].values.tolist()
list_.reverse()
for l in list_:
    print("Score = %1.5f \n"% l[1], l[2],'\n',l[0])

Score = 0.91133 
 [0.8660353636691782, 0.8801092590398016, 0.9034511713381252, 0.9113272788204542, 0.8957136780039177, 0.900470811885921, 0.8960027454910271] 
 {'batch_size': 128,
 'class_ratio': [0.7, 3, 2.5, 1.5],
 'epoch': [5, 10],
 'gamma': 0.1,
 'learning_rate': 1e-05,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 5,
 'transforms': [1, None],
 'val_ratio': [0.03, 0.15, 0.15, 0.2],
 'weight_decay': 0.001}

Score = 0.90644 
 [0.8918884050049516, 0.8988137298080587, 0.8762885356466329, 0.896869212091259, 0.9064381767101568, 0.885811101204673, 0.8888843661715389, 0.8914545646585141] 
 {'batch_size': 128,
 'class_ratio': [0.8, 2, 2, 2],
 'epoch': [5, 10],
 'gamma': 0.3,
 'learning_rate': 0.001,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 5,
 'transforms': [1, None],
 'val_ratio': [0.03, 0.15, 0.15, 0.2],
 'weight_decay': 0.001}

Score = 0.90136 
 [0.8714103816093376, 0.9013643892799675, 0.8913542884781588, 0.8689

In [13]:
list_ = df[df['model']=='EfficientNet'][['config','best_score','score']].iloc[-3:].values.tolist()
list_.reverse()
for l in list_:
    print("Score = %1.5f \n"% l[1], l[2],'\n',l[0])

In [14]:
list_ = df[df['model']=='VGG16'][['config','best_score']].iloc[:3].values.tolist()
for l in list_:
    print("Score = %1.5f \n"% l[1],l[0])

df[df['model']=='VGG16'][['best_monotone','best_screenshot', 'best_unknown']].iloc[:3]
    

Score = 0.87142 
 {'batch_size': 128,
 'class_ratio': [0.7, 3, 2.5, 1.5],
 'epoch': [5, 10],
 'gamma': 0.1,
 'learning_rate': 0.001,
 'loss_cls_weight': tensor([1.1000, 1.0000, 1.0000, 1.1000]),
 'optimizer': 'Adam',
 'step_size': 5,
 'transforms': [1, None],
 'val_ratio': [0.03, 0.15, 0.15, 0.2],
 'weight_decay': 0}

Score = 0.88842 
 {'batch_size': 128,
 'class_ratio': [0.7, 3, 2.5, 1.5],
 'epoch': [5, 10],
 'gamma': 0.3,
 'learning_rate': 0.0001,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 2,
 'transforms': [1, None],
 'val_ratio': [0.03, 0.15, 0.15, 0.2],
 'weight_decay': 0.001}

Score = 0.90457 
 {'batch_size': 128,
 'class_ratio': [0.7, 3, 2.5, 1.5],
 'epoch': [2, 15],
 'gamma': 0.1,
 'learning_rate': 0.0001,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 5,
 'transforms': [1, None],
 'val_ratio': [0.03, 0.15, 0.15, 0.2],
 'weight_decay': 0}



,best_monotone,best_screenshot,best_unknown
18,0.918367,0.899263,0.931252
20,0.944724,0.938462,0.924345
19,0.947368,0.946292,0.934105


In [15]:
list_ = df[df['model']=='ResNet50'][['config','best_score']].iloc[:3].values.tolist()
for l in list_:
    print("Score = %1.5f \n"% l[1],l[0])

df[df['model']=='VGG16'][['best_monotone','best_screenshot', 'best_unknown']].iloc[:3]
    

Score = 0.86374 
 {'batch_size': 128,
 'class_ratio': [0.7, 3, 2.5, 1.5],
 'epoch': [5, 10],
 'gamma': 0.3,
 'learning_rate': 1e-05,
 'loss_cls_weight': tensor([1., 1., 1., 1.]),
 'optimizer': 'Adam',
 'step_size': 2,
 'transforms': [1, None],
 'val_ratio': [0.03, 0.15, 0.15, 0.2],
 'weight_decay': 0.001}

Score = 0.88134 
 {'batch_size': 128,
 'class_ratio': [0.8, 2, 2, 2],
 'epoch': [2, 15],
 'gamma': 0.1,
 'learning_rate': 0.001,
 'loss_cls_weight': tensor([1.1000, 1.0000, 1.0000, 1.1000]),
 'optimizer': 'Adam',
 'step_size': 2,
 'transforms': [2,
                Compose(
    ColorJitter(brightness=None, contrast=None, saturation=[0.95, 1.05], hue=[-0.05, 0.05])
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)],
 'val_ratio': [0.03, 0.15, 0.15, 0.2],
 'weight_decay': 0}

Score = 0.89118 
 {'batch_size': 128,
 'class_ratio': [0.7, 3, 2.5, 1.5],
 'epoch': [2, 15],
 'gamma': 0.3,
 'learning_rate': 0.0001,
 'loss_cls_w

,best_monotone,best_screenshot,best_unknown
18,0.918367,0.899263,0.931252
20,0.944724,0.938462,0.924345
19,0.947368,0.946292,0.934105
